# Getting Started

Welcome to **PyTorch-Ignite**’s quick start guide that covers the
essentials of getting a project up and running while walking through
basic concepts of Ignite. In just a few lines of code, you can get your
model trained and validated. The complete code can be found at the end
of this guide.

## Prerequisites

This tutorial assumes you are familiar with the:

1.  Basics of Python and deep learning
2.  Structure of PyTorch code

## Installation

1.  From `pip`

``` shell
pip install pytorch-ignite
```

1.  From `conda`

``` shell
conda install ignite -c pytorch
```

See [here](https://pytorch-ignite.netlify.app/docs/how-to-guides/installation) for other installation
options.

## Code

Import the following:

In [13]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.models import resnet18
from torchvision.transforms import Compose, Normalize, ToTensor

from ignite.engine import Engine, Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.contrib.handlers.tqdm_logger import ProgressBar

Speed things up by setting [device](https://pytorch.org/docs/stable/tensor_attributes.html#torch.torch.device) to `cuda` if available else `cpu`.

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Define a class of your model or use the predefined ResNet18 model (modified for MNIST) below, instantiate it and move it to device:

In [5]:
class Net(nn.Module):

    def __init__(self, in_channels=1):
        super(Net, self).__init__()
        
        # Changed the output layer to output 10 classes instead of 1000 classes
        self.model = resnet18(num_classes=10)

        # Changed the input layer to take grayscale images for MNIST instaed of RGB images
        self.model.conv1 = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(3, 3), bias=False)


    def forward(self, x):
        return self.model(x)

model = Net().to(device)

Now let us define the training and validation datasets (as
[torch.utils.data.DataLoader](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader))
and store them in `train_loader` and `val_loader` respectively. We have
used the [MNIST](https://pytorch.org/vision/stable/datasets.html#mnist)
dataset for ease of understanding.


In [6]:
data_transform = Compose([ToTensor(), Normalize((0.1307,), (0.3081,))])

train_loader = DataLoader(
    MNIST(download=True, root=".", transform=data_transform, train=True), batch_size=128, shuffle=True
)

val_loader = DataLoader(
    MNIST(download=True, root=".", transform=data_transform, train=False), batch_size=256, shuffle=False
)


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw




Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw




Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Finally, we will specify the optimizer and the loss function:

In [7]:
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.005)
criterion = nn.CrossEntropyLoss()

And we’re done with setting up the important parts of the project.
PyTorch-Ignite will handle all other boilerplate code as we will see
below. Next we have to define a trainer engine by passing our model,
optimizer and loss function to
[`create_supervised_trainer`](https://pytorch.org/ignite/generated/ignite.engine.create_supervised_trainer.html),
and an evaluator engine by passing Ignite’s out-of-the-box
[metrics](https://pytorch.org/ignite/metrics.html#complete-list-of-metrics)
and the model to
[`create_supervised_evaluator`](https://pytorch.org/ignite/v0.4.5/generated/ignite.engine.create_supervised_evaluator.html#create-supervised-evaluator)
:

In [8]:
trainer = create_supervised_trainer(model, optimizer, criterion, device)

val_metrics = {
    "accuracy": Accuracy(),
    "nll": Loss(criterion)
}
evaluator = create_supervised_evaluator(model, metrics=val_metrics, device=device)

Both `trainer` and `evaluator` objects are instances of
[`Engine`](https://pytorch.org/ignite/v0.4.5/generated/ignite.engine.engine.Engine.html#ignite.engine.engine.Engine) - the main component of Ignite, which is essentially an abstraction over
the training or validation loop.

If you need more control over your training and validation loops, you
can create custom `trainer` and `evaluator` objects by wrapping the step
logic in `Engine` :

```python
def train_step(engine, batch):
    model.train()
    optimizer.zero_grad()
    x, y = batch[0].to(device), batch[1].to(device)
    y_pred = model(x)
    loss = criterion(y_pred, y)
    loss.backward()
    optimizer.step()
    return loss.item()

trainer = Engine(train_step)

def validation_step(engine, batch):
    model.eval()
    with torch.no_grad():
        x, y = batch[0].to(device), batch[1].to(device)
        y_pred = model(x)
        return y_pred, y

evaluator = Engine(validation_step)

# Attach metrics to the evaluator
for name, metric in val_metrics.items():
    metric.attach(evaluator, name)
```

We can customize the code further by adding all kinds of event handlers.
`Engine` allows adding handlers on various events that are triggered
during the run. When an event is triggered, attached handlers
(functions) are executed. Thus, for logging purposes we add a function
to be executed at the end of every `log_interval`-th iteration:

In [9]:
# How many batches to wait before logging training status
log_interval=100

In [10]:
@trainer.on(Events.ITERATION_COMPLETED(every=log_interval))
def log_training_loss(engine):
    print(f"Epoch[{engine.state.epoch}] Loss: {engine.state.output:.2f}")

or equivalently without the decorator but attaching the handler function
to the `trainer` via
[`add_event_handler`](https://pytorch.org/ignite/v0.4.5/generated/ignite.engine.engine.Engine.html#ignite.engine.engine.Engine.add_event_handler)

``` python
def log_training_loss(engine):
    print(f"Epoch[{engine.state.epoch}] Loss: {engine.state.output:.2f}")

trainer.add_event_handler(Events.ITERATION_COMPLETED, log_training_loss)
```

After an epoch ends during training, we can compute the training and
validation metrics by running `evaluator` on `train_loader` and
`val_loader`. Hence we will attach two additional handlers to `trainer`
when an epoch completes:

In [11]:
@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    evaluator.run(train_loader)
    metrics = evaluator.state.metrics
    print(f"Training Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['nll']:.2f}")

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    evaluator.run(val_loader)
    metrics = evaluator.state.metrics
    print(f"Validation Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['nll']:.2f}")

We can even attach a TQDM sytle [`ProgressBar()`](https://pytorch.org/ignite/v0.4.5/generated/ignite.contrib.handlers.tqdm_logger.html#ignite.contrib.handlers.tqdm_logger.ProgressBar) to the trainer for logging training progress and even computed metrics.

In [14]:
pbar = ProgressBar()
pbar.attach(trainer)

Finally, we start the engine on the training dataset and run it for 5
epochs:

In [15]:
trainer.run(train_loader, max_epochs=5)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch[1] Loss: 0.34
Epoch[1] Loss: 0.12
Epoch[1] Loss: 0.12
Epoch[1] Loss: 0.17
Training Results - Epoch[1] Avg accuracy: 0.94 Avg loss: 0.20
Validation Results - Epoch[1] Avg accuracy: 0.94 Avg loss: 0.20


Epoch[2] Loss: 0.11
Epoch[2] Loss: 0.26
Epoch[2] Loss: 0.02
Epoch[2] Loss: 0.13
Epoch[2] Loss: 0.05
Training Results - Epoch[2] Avg accuracy: 0.97 Avg loss: 0.16
Validation Results - Epoch[2] Avg accuracy: 0.97 Avg loss: 0.21


Epoch[3] Loss: 0.05
Epoch[3] Loss: 0.08
Epoch[3] Loss: 0.13
Epoch[3] Loss: 0.03
Epoch[3] Loss: 0.07
Training Results - Epoch[3] Avg accuracy: 0.99 Avg loss: 0.04
Validation Results - Epoch[3] Avg accuracy: 0.99 Avg loss: 0.05


Epoch[4] Loss: 0.10
Epoch[4] Loss: 0.15
Epoch[4] Loss: 0.02
Epoch[4] Loss: 0.02
Training Results - Epoch[4] Avg accuracy: 0.97 Avg loss: 0.13
Validation Results - Epoch[4] Avg accuracy: 0.96 Avg loss: 0.15


Epoch[5] Loss: 0.03
Epoch[5] Loss: 0.05
Epoch[5] Loss: 0.00
Epoch[5] Loss: 0.02
Epoch[5] Loss: 0.09
Training Results - Epoch[5] Avg accuracy: 0.99 Avg loss: 0.03
Validation Results - Epoch[5] Avg accuracy: 0.99 Avg loss: 0.04


State:
	iteration: 2345
	epoch: 5
	epoch_length: 469
	max_epochs: 5
	output: 0.010196101851761341
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

## Next Steps

1.  Check out [tutorials](/docs/tutorials) if you want to continue
    learning more about PyTorch-Ignite.
2.  Head over to [how-to guides](/docs/how-to-guides) if you’re looking
    for a specific problem.
3.  If you want to set-up a PyTorch-Ignite project, visit [Code
    Generator](https://code-generator.netlify.app/) to get a variety of
    easily customizable templates and out-of-the-box features.

## Complete Code

``` python
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.models import resnet18
from torchvision.transforms import Compose, Normalize, ToTensor

from ignite.engine import Engine, Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.contrib.handlers.tqdm_logger import ProgressBar

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Net(nn.Module):

    def __init__(self, in_channels=1):
        super(Net, self).__init__()
        
        # Changed the output layer to output 10 classes instead of 1000 classes
        self.model = resnet18(num_classes=10)

        # Changed the input layer to take grayscale images for MNIST instaed of RGB images
        self.model.conv1 = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(3, 3), bias=False)


    def forward(self, x):
        return self.model(x)

model = Net().to(device)

data_transform = Compose([ToTensor(), Normalize((0.1307,), (0.3081,))])

train_loader = DataLoader(
    MNIST(download=True, root=".", transform=data_transform, train=True), batch_size=128, shuffle=True
)

val_loader = DataLoader(
    MNIST(download=True, root=".", transform=data_transform, train=False), batch_size=256, shuffle=False
)

optimizer = torch.optim.RMSprop(model.parameters(), lr=0.005)
criterion = nn.CrossEntropyLoss()

trainer = create_supervised_trainer(model, optimizer, criterion, device)

val_metrics = {
    "accuracy": Accuracy(),
    "nll": Loss(criterion)
}
evaluator = create_supervised_evaluator(model, metrics=val_metrics, device=device)

# how many batches to wait before logging training status
log_interval=20

@trainer.on(Events.ITERATION_COMPLETED(every=log_interval))
def log_training_loss(engine):
    print(f"Epoch[{engine.state.epoch}] Loss: {engine.state.output:.2f}")

@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    evaluator.run(train_loader)
    metrics = evaluator.state.metrics
    print(f"Training Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['nll']:.2f}")

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    evaluator.run(val_loader)
    metrics = evaluator.state.metrics
    print(f"Validation Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['nll']:.2f}")

pbar = ProgressBar()
pbar.attach(trainer)

trainer.run(train_loader, max_epochs=5)
```